## Import libraries

In [162]:
import dash # initialize your application
import dash_core_components as dcc # allows you to create interactive components like graphs, dropdowns, or date ranges
import dash_html_components as html # access HTML tags
import pandas as pd
import numpy as np

from datetime import datetime
from dash.dependencies import Output, Input

## Import data

In [310]:
data = pd.read_csv("Swim_dataset.csv") # read avocado csv file

# filter the unnecessary columns
unnec_cols = data.columns[10:]
data = data.drop(columns=unnec_cols)

# rename "Unnamed: 0" column name to "Date"
data = data.rename(columns={"Unnamed: 0":"Date"})

# convert the contents in the date column into a datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d") # convert the date into a date format

# set Date as an index 
# data = data.set_index('Date')

# convert nan to 0
data = data.fillna(0)

data.head()

,Date,Warm Up,Kick,Pull,Swim,Drill,Main Set,Post Set,Cool Down,Total
0,2020-01-02,400.0,600.0,0.0,600.0,400.0,2800.0,100.0,400.0,5300
1,2020-01-03,400.0,0.0,0.0,1600.0,400.0,2000.0,600.0,400.0,5400
2,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2020-01-06,800.0,0.0,0.0,2200.0,0.0,2500.0,880.0,200.0,6580


#### 1 day

In [315]:
# 1일치
one_day_data = data.iloc[0][1:-1]

# columns
categories = data.columns[1:-1]

#### 1 month (total)

In [316]:
start_date = '2020-01-02'
end_date = '2020-02-02'

mask = (
        (data.Date >= start_date)
        & (data.Date <= end_date)
    )

filtered_data = data.loc[mask, :]

column_list = filtered_data.columns[1:]
total_time_list = []
for column in column_list:
    category_list = list(filtered_data[column])
    total_time = sum(category_list)
    total_time_list.append(total_time)

total_time_list

[14200.0, 10900.0, 12400.0, 18120.0, 3200.0, 82950.0, 3280.0, 7100.0, 152150]

#### 1 month (average)

In [317]:
start_date = '2020-01-02'
end_date = '2020-02-02'

start_date_time = datetime.strptime(start_date, '%Y-%m-%d')
end_date_time = datetime.strptime(end_date, '%Y-%m-%d')

days = (end_date_time - start_date_time).days

avg_time_list = []
for i in range(len(total_time_list)):
    avg_time_list.append(total_time_list[i] / days)

avg_time_list

[458.06451612903226,
 351.61290322580646,
 400.0,
 584.516129032258,
 103.2258064516129,
 2675.8064516129034,
 105.80645161290323,
 229.03225806451613,
 4908.064516129032]

## Initialize the application

In [318]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "Swimming Training Time Analysis"

## App layout

In [319]:
graph_bgcolor = "#0A165A"
font_color = "#FFFFFF"

In [330]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.H1(
                    children="Swimming Training Analysis", className="header-title"
                ),
                html.P(
                    children="Analyze the how the training time affects the result",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        
        # Graph part
        html.Div(
            children=[
                
                # Competition graph
                html.Div(
                    children=[
                        
                        # title
                        html.Div(children='Competition Graph', className='competition-title'),
                        
                        # menu
                        html.Div(
                            children=[
                                
                                # average-total selection
                                dcc.RadioItems(
                                    id='competition-avg-total-selector',
                                    options=[
                                        {'label': 'Average', 'value': 'avg'},
                                        {'label': 'Total', 'value': 'total'},
                                    ],
                                    value='avg',
                                    labelStyle={
                                        'display': 'flex',
                                        'margin-bottom': '5px',
                                        'font-weight': 300,
                                        'color': '#FFFFFF',
                                    },
                                ),
                                
                                # category selector
                                dcc.Dropdown(
                                    id='competition-category-selector',
                                    options=[
                                        {'label': 'Warm Up', 'value': 'Warm Up'},
                                        {'label': 'Kick', 'value': 'Kick'},
                                        {'label': 'Pull', 'value': 'Pull'},
                                        {'label': 'Swim', 'value': 'Swim'},
                                        {'label': 'Drill', 'value': 'Drill'},
                                        {'label': 'Main Set', 'value': 'Main Set'},
                                        {'label': 'Post Set', 'value': 'Post Set'},
                                        {'label': 'Cool Down', 'value': 'Cool Down'},
                                    ],
                                    multi=True,
                                    style={"min-width": "200px"},
                                ),
                                
                                # date
                                dcc.DatePickerRange(
                                    id='date-range-selector',
                                    min_date_allowed=data.Date.min().date(),
                                    max_date_allowed=data.Date.max().date(),
                                    start_date=data.Date.min().date(),
                                    end_date=data.Date.max().date(),
                                ),
                            ],
                            className='menu'
                        ),
                        
                        # graph
                        dcc.Graph(
                            figure={
                                "data": [
                                    {
                                        "x": categories,
                                        "y": avg_time_list,
                                        "type": "bar",
                                        "name": "1 month data",
                                    },
                                ],
                                "layout": {
                                    "paper_bgcolor": graph_bgcolor,
                                    "plot_bgcolor": graph_bgcolor,
                                    "font": {
                                        "color": "#FFFFFF"
                                    }
                                },
                            },
                            className='competition-graph',
                        ),
                    ],
                    className='competition-graph-container'
                ),
                
                
                # Training graph
                html.Div(
                    children=[
                        
                        # categorical bar plot
                        html.Div(
                            children=[
                                # title
                                html.Div(children='Categorical Graph', className='categorical-graph-title'),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='category-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='category-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            value="Warm Up",
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),

                                        # date
#                                         dcc.DatePickerRange(
#                                             id='category-date-range-selector',
#                                             min_date_allowed=data.Date.min().date(),
#                                             max_date_allowed=data.Date.max().date(),
#                                             start_date=data.Date.min().date(),
#                                             end_date=data.Date.max().date(),
#                                             className='datepicker',
#                                         ),
                                    ],
                                    className='train-menu',
                                ),

                                # bar plot
                                html.Div(
                                    children=dcc.Graph(
                                        figure={
                                            "data": [
                                                {
                                                    "x": categories,
                                                    "y": avg_time_list,
                                                    "type": "bar",
                                                    "name": "1 month data",
                                                },
                                            ],
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                        style={"height": "100%"},
                                    ),
                                    className='bar-categorical-graph'
                                ),
                            ],
                            className='bar-categorical-graph-container'
                        ),
                        
                        # Categorical line plot
                        html.Div(
                            children=[
                                # title
                                html.Div(children='Time Series Graph', className='time-graph-title'),

                                # menu
                                html.Div(
                                    children=[
                                        # average-total selection
                                        dcc.RadioItems(
                                            id='time-avg-total-selector',
                                            options=[
                                                {'label': 'Average', 'value': 'avg'},
                                                {'label': 'Total', 'value': 'total'},
                                            ],
                                            value='avg',
                                            labelStyle={
                                                'display': 'flex',
                                                'margin-bottom': '5px',
                                                'font-weight': 300,
                                                'color': '#FFFFFF',
                                            },
                                        ),

                                        # category selector
                                        dcc.Dropdown(
                                            id='time-category-selector',
                                            options=[
                                                {'label': 'Warm Up', 'value': 'Warm Up'},
                                                {'label': 'Kick', 'value': 'Kick'},
                                                {'label': 'Pull', 'value': 'Pull'},
                                                {'label': 'Swim', 'value': 'Swim'},
                                                {'label': 'Drill', 'value': 'Drill'},
                                                {'label': 'Main Set', 'value': 'Main Set'},
                                                {'label': 'Post Set', 'value': 'Post Set'},
                                                {'label': 'Cool Down', 'value': 'Cool Down'},
                                            ],
                                            multi=True,
                                            style={"min-width": "150px"},
                                        ),

                                        # date
#                                         dcc.DatePickerRange(
#                                             id='time-date-range-selector',
#                                             min_date_allowed=data.Date.min().date(),
#                                             max_date_allowed=data.Date.max().date(),
#                                             start_date=data.Date.min().date(),
#                                             end_date=data.Date.max().date(),
#                                             className='datepicker',
#                                         ),
                                    ],
                                    className='train-menu'
                                ),

                                # line plot
                                html.Div(
                                    children=dcc.Graph(
                                        figure={
                                            "data": [
                                                {
                                                    "x": categories,
                                                    "y": avg_time_list,
                                                    "type": "bar",
                                                    "name": "1 month data",
                                                },
                                            ],
                                            "layout": {
                                                "paper_bgcolor": graph_bgcolor,
                                                "plot_bgcolor": graph_bgcolor,
                                                "font": {
                                                    "color": "#FFFFFF"
                                                }
                                            },
                                        },
                                        style={"height": "100%"},
                                    ),
                                    className='time-series-graph'
                                ),
                            ],
                            className='time-series-graph-container',
                        ),
                    ],
                    className='training-graph-container',
                ),
            ],
            className='graph-container'
        ),
        
        # space
        html.Div(style={"height": 80}),
    ],
    className='app-container'
)

## Competition graph callback

In [331]:
# @app.callback(
#     [Output("competition-graph", "figure")]
# )

## Categorical bar chart callback

In [328]:
@app.callback(
    [Output("categorical-bar-chart", "figure")],
    [
        Input("category-avg-total-selector", "value"),
        Input("category-category-selector", "value"),
        Input("date-range-selector", "start_date"),
        Input("date-range-selector", "end_date"),
    ],
)
def update_bar_chart(avg_or_total, categories, start_date, end_date):
    # filter by date
#     date_mask = (
#         (data.Date >= start_date)
#         & (data.Date <= end_date)
#     )
#     new_data = data.loc[date_mask, :]

#     # filter by categories
#     new_data = new_data[categories]
    
    
#     # get avg or total data
#     total_data = []
#     for category in categories:
#         category_list = list(new_data[category])
#         total_value = sum(category_list)
#         total_data.append(total_value)
    
#     if avg_or_total == 'avg':
#         for i in range(len(total_data)):
#             total_data[i] /= new_data.shape[0]
            
    bar_chart_figure = {
        "data": [
            {
                "x": ["Warm Up"],
                "y": [3900],
                "type": "bar",
                "name": "1 month data",
            },
        ],
        "layout": {
            "paper_bgcolor": graph_bgcolor,
            "plot_bgcolor": graph_bgcolor,
            "font": {
                "color": "#FFFFFF"
            }
        },
    }
    
    return bar_chart_figure

In [ ]:
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Sep/2021 15:45:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 15:45:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 15:45:10] "GET /_dash-dependencies HTTP/1.1" 200 -
